In [1]:
from gensim.test.utils import datapath
from gensim import utils
from gensim.models import Word2Vec
import gensim.downloader as api
import pprint
import tqdm

pp = pprint.PrettyPrinter()

## train by gensim

In [2]:
print(api.info("text8"))
dataset = api.load("text8")

{'num_records': 1701, 'record_format': 'list of str (tokens)', 'file_size': 33182058, 'reader_code': 'https://github.com/RaRe-Technologies/gensim-data/releases/download/text8/__init__.py', 'license': 'not found', 'description': 'First 100,000,000 bytes of plain text from Wikipedia. Used for testing purposes; see wiki-english-* for proper full Wikipedia datasets.', 'checksum': '68799af40b6bda07dfa47a32612e5364', 'file_name': 'text8.gz', 'read_more': ['http://mattmahoney.net/dc/textdata.html'], 'parts': 1}


In [3]:
dataset = open("dataset/text8_300.txt").read()
model = Word2Vec(dataset)

In [4]:
print(model.wv['king'])

[-1.5646565   2.1416721  -1.1539199  -1.9851359   2.619045   -3.3444808
  0.959632   -0.6802612  -0.41247857 -0.5359626   0.56075853  0.32990667
 -3.378159   -0.62457     0.35390902 -0.6240219  -1.0453948  -0.24845748
 -1.2887543   0.50618345  0.81862074 -2.1014953   0.675516   -0.5611012
  2.0409162  -0.01887369 -1.7865096  -0.670666   -1.5321208   0.9704901
  3.0289795   1.3251785  -0.17843336  0.7755415  -0.30679053 -1.4795413
 -2.012141    4.0937805  -0.6525124  -2.5100193  -1.184536    1.542913
 -0.56440574  2.8577325  -1.153359   -0.6450355   0.9970867   1.010052
 -2.4158788   1.1906601   1.1570123   1.1849813  -0.12843601  1.2656046
  0.92131644  0.3270518  -0.42982298  1.886766    1.4391572  -2.0707736
 -0.75522614 -0.6225984  -1.0147653   1.1164544  -3.337238    1.0459816
 -1.2972715   2.9087422  -2.2818244   0.6408788  -2.0624948   1.3867028
 -1.4881582   0.01291318 -1.9770612  -1.565      -1.4328805   0.42885882
 -1.5217371   1.8176492   2.5530162  -2.1131124   3.5719972   0

In [5]:
print(model.wv.evaluate_word_pairs(datapath('wordsim353.tsv')))
print(model.wv.evaluate_word_pairs(datapath('simlex999.txt')))

((0.605667297981981, 1.6248948064593337e-36), SpearmanrResult(correlation=0.6233151633189905, pvalue=3.606318407136446e-39), 0.56657223796034)
((0.27664388334780876, 6.931294310016631e-19), SpearmanrResult(correlation=0.25681873491079726, pvalue=2.0905245648312486e-16), 0.7007007007007007)


## train by word2vec diy

In [3]:
dataset = api.load("text8")
with open("dataset/text8.txt", "w") as f:
    for data in dataset:
        f.write(" ".join(data) + "\n")

In [ ]:
from gensim.test.utils import datapath
from gensim import utils
from gensim.models import Word2Vec
import gensim.downloader as api
import pprint
import tqdm

from skip_gram_pytorch.word2vec2 import word2vec
pp = pprint.PrettyPrinter()
%load_ext autoreload
%autoreload 2

# wv = word2vec("dataset/text8_300.txt") # emb_dim=100, vocab=30000, SparseAdam, lr=0.001
wv = word2vec("dataset/text8.txt")
wv.train()

len data: 17005207
eporch= 1, batch=290000: sp=0.562 0.256  pair/sec = 10076.14 loss=0.71366


In [2]:
from word2vec.trainer import Word2VecTrainer

%load_ext autoreload
%autoreload 2
# corpus_path = datapath('lee_background.cor')
# corpus_path = "dataset/text8.txt"
corpus_path = "dataset/text8_100.txt"
w2v = Word2VecTrainer(input_file=corpus_path, output_file="out.vec")
print("Training...")
w2v.train()
print("Train Done.")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Read 1M words.
Total embeddings: 7328


  0%|          | 0/4 [00:00<?, ?it/s]

Training...

Epoch: 1


  0%|          | 0/4 [00:04<?, ?it/s]


RuntimeError: Input, output and indices must be on the current device

In [3]:
from gensim.models import KeyedVectors

wv = KeyedVectors.load_word2vec_format("out.vec", binary=False)
vocab = list(wv.vocab.keys())
print("Loaded vocab size %i" % len(vocab))

Loaded vocab size 15446


In [4]:
print(wv.evaluate_word_pairs(datapath('wordsim353.tsv')))
print(wv.evaluate_word_pairs(datapath('simlex999.txt')))

((0.10070935787561462, 0.08263074811110695), SpearmanrResult(correlation=0.06339930418941554, pvalue=0.2752993468521408), 15.58073654390935)
((-0.00834582020339468, 0.8148234262022849), SpearmanrResult(correlation=-0.02203231446812901, pvalue=0.5363410818519248), 20.92092092092092)
